## Imports

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import math
import re

from util.word2vec_as_MF import Word2vecMF
from util.functions import *

import time
dimension = 100
regularization =0.0
model_enwik = Word2vecMF()
negative = 5

Please run:
  python setup.py build_ext --inplace


## Read and preprocess enwik9

# Gensim

In [2]:
'''%%time
skip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'''

'%%time\nskip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'

In [3]:
'''skip.get_latest_training_loss()'''

'skip.get_latest_training_loss()'

## Train ro_sgns model starting from SVD of SPPMI

In [4]:
%%time
# Create word2vobject has no attributeec as matrix factorization model

'''model_enwik.data_to_matrices(real_sentences, dimension, 5, 
                             DB_to_file='enwik-2            try:
00/matrices.npz',
                            indices_to_file='enwik-200/vocab.txt')'''
#sentences = load('data/x1')
#model_enwik.load_vocab('v1.txt')
model_enwik.data_to_matrices('data/x1', r=dimension, k=negative,
                             DB_to_file=False,
                            vocab_to_file='v1.txt')
#model_enwik.save_vocabulary(from_file='data/x1',vocab_to_file='v.txt', r=100)


Parsing sentences from training set
Length of vocabulary is: 356
CPU times: user 578 ms, sys: 7.87 ms, total: 585 ms
Wall time: 582 ms


In [5]:
#C0, W0, step_size = BFGD_init(model_enwik, dimension=dimension, reg=regularization)
C0, W0 = SPPMI_init(model_enwik, dimension=dimension, negative = negative)
#C0, W0, step_size = RAND_init(model_enwik, dimension=dimension)
model_enwik.save_CW('initializations/NYT',0)
print(np.count_nonzero(C0))

0.047784370660270165
62.27460890120043
('reg ', 1.0575829573654522e-13)
('Initial loss', 1808805.0213349368)
35600


/home/yerong/local/Anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [6]:
print(C0.shape[1])
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='AM', 
               d=dimension,
               eta = 5e-6,
               lbd = 1.0,
               MAX_ITER=189000,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

356


'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'AM\', \n               d=dimension,\n               eta = 5e-6,\n               lbd = 1.0,\n               MAX_ITER=189000,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [7]:
start_time = time.time()
opt_experiment(model_enwik,
               mode='BFGD',
               d=dimension,
               eta=1e-5,
               #eta = step_size,
               reg = regularization,
               MAX_ITER=50,
               from_iter=0,
               start_from='test',
               itv_print=1,
               itv_save=1,
               init=(True, C0, W0), display=True,
               autostop=False)
print("--- %s seconds ---" % (time.time() - start_time))

enwik-200/BFGDiter_fromtest_dim100_step1e-05_0.0
('reg ', 1.0575829573654522e-13)
('Iter #:', 0, 'loss', 1808805.0213349368)
('reg ', 0.015319033869243501)
('Iter #:', 1, 'loss', 1807098.5038878059)
('reg ', 0.025543132619958855)
('Iter #:', 2, 'loss', 1805833.1888186142)
('reg ', 0.03280857371592869)
('Iter #:', 3, 'loss', 1804774.8047999986)
('reg ', 0.03888353358583777)
('Iter #:', 4, 'loss', 1803731.2341214044)
('reg ', 0.04568187901735637)
('Iter #:', 5, 'loss', 1802516.6522106032)
('reg ', 0.05574735316134854)
('Iter #:', 6, 'loss', 1800919.5783693437)
('reg ', 0.0725169358502605)
('Iter #:', 7, 'loss', 1798671.0158157595)
('reg ', 0.1002695394697543)
('Iter #:', 8, 'loss', 1795412.8785928302)
('reg ', 0.1440951869136372)
('Iter #:', 9, 'loss', 1790674.0421115258)
('reg ', 0.20999008835071908)
('Iter #:', 10, 'loss', 1783874.3028980736)
('reg ', 0.3044683969766074)
('Iter #:', 11, 'loss', 1774393.5644365659)
('reg ', 0.4329830545841495)
('Iter #:', 12, 'loss', 1761746.1112003534)